In [51]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import requests

In [ ]:
from config import gkey
import gmaps
gmaps.configure(api_key=gkey)

In [52]:
shootings_df = pd.read_csv("Stanford_MSA_Shooting.csv")
shootings_df.head()

,CaseID,Title,Location,City,State,Latitude,Longitude,Number of Civilian Fatalities,Number of Civilian Injured,Number of Enforcement Fatalities,...,Data Source 3,Data Source 4,Data Source 5,Data Source 6,Data Source 7,Military Experience,Class,Depreciation,Notes,Edit Date
0,1,University of Texas at Austin,"Austin, Texas",Austin,Texas,30.198887,-97.844159,15,32,1,...,http://news.google.com/newspapers?id=lkk0AAAAI...,http://news.google.com/newspapers?id=PPUjAAAAI...,http://books.google.com/books?id=ClYEAAAAMBAJ&...,NaN,NaN,Yes,SPK,1,NaN,6/8/2016
1,2,Rose-Mar College of Beauty,"Mesa, Arizona",Mesa,Arizona,33.422687,-111.816320,5,1,0,...,http://www.nydailynews.com/news/crime/beauty-s...,http://books.google.com/books?id=Cre7qsswRiwC&...,NaN,NaN,NaN,Unknown,MS,1,NaN,6/8/2016
2,3,New Orleans Police Shootings,"New Orleans, Louisiana",New Orleans,Louisiana,30.068724,-89.931474,4,8,5,...,http://www.trutv.com/library/crime/notorious_m...,http://books.google.com/books?id=TfEDmROcZwEC&...,NaN,NaN,NaN,Unknown,SPK,1,NaN,6/22/2016
3,4,Clara Barton Elementary School,"Chicago, Illinois",Chicago,Illinois,41.839280,-87.688181,1,3,0,...,http://www.leagle.com/xmlResult.aspx?page=1&xm...,NaN,NaN,NaN,NaN,Unknown,MS,1,NaN,6/21/2016
4,5,Olean High School,"Olean, New York",Olean,New York,42.081854,-78.432139,3,7,0,...,http://www.newswithviews.com/Erica/Carle10.htm,http://books.google.com/books?id=ZuKoSskEWyIC&...,"""Olean High School Shooting"". Larrie Benton Za...",NaN,NaN,Unknown,MS,1,NaN,6/21/2016


In [53]:
#ANGIE
#Filter out needed columns from original dataframe
shootings_df = shootings_df[["Title", "City", "State", "Latitude", "Longitude", "Number of Civilian Fatalities",
                    "Number of Civilian Injured", "Number of Enforcement Fatalities", 
                    "Number of Enforcement Injured","Total Number of Victims",
                    "Date", "Type of Gun - General",
                    "Total Number of Guns", "Relationship to Incident Location"]]

In [ ]:
#ANGIE
#Obtain coordinates from the latitude and longitude columns
loc = []
for index, row in shootings_df.iterrows():
    coordinates = (row["Latitude"],row["Longitude"]) 
    loc.append(coordinates)

In [ ]:
#ANGIE
#Format map
figure_layout = {
    'width': '800px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [ ]:
#ANGIE
#Assign the marker layer to a variable
markers = gmaps.marker_layer(loc)
#Add the layer to the map
fig.add_layer(markers)
fig

In [54]:
#ANGIE
#Data cleaning
shootings_df["Relationship to Incident Location"] = shootings_df["Relationship to Incident Location"].replace(
    {"Place of Residency": "Place of residency", "Place of Recreation": "Place of recreation",
    "Place of Business/employment": "Place of business/employment", "Residential home/Neighborhood": "Place of residency" })

shootings_df["Type of Gun - General"] = shootings_df["Type of Gun - General"].replace({"handgun": "Handgun", "Multiple Guns": "Multiple guns",
                                                                                      "Multiple guns\r\n": "Multiple guns", "\r\nMultiple guns": "Multiple guns",
                                                                                      "Semi-Automatic Rifle": "Rifle", "9-mm": "Handgun"})

#Fill empty rows with unknown
shootings_df["Relationship to Incident Location"] = shootings_df["Relationship to Incident Location"].fillna("Unknown")


In [55]:
#ANGIE
#shootings_df.count()
#shootings_df["Relationship to Incident Location"].value_counts()
#shootings_df["Type of Gun - General"].value_counts()

In [56]:
#ANGIE
#Combine the lat. and long. into one column
shootings_df["Location"] = (shootings_df["Latitude"]).astype(str)+','+ shootings_df["Longitude"].astype(str)

#Create new column with total number of victim fatalitites
shootings_df["Total Number of Victim Fatalities"] = shootings_df["Number of Civilian Fatalities"] + shootings_df["Number of Enforcement Fatalities"]

#Rearrange column order
shootings_df = shootings_df[["Title", "City", "State", "Location", "Number of Civilian Fatalities",
                    "Number of Civilian Injured", "Number of Enforcement Fatalities", 
                    "Number of Enforcement Injured", "Total Number of Victim Fatalities", "Total Number of Victims",
                    "Date", "Type of Gun - General",
                    "Total Number of Guns", "Relationship to Incident Location"]]
shootings_df


,Title,City,State,Location,Number of Civilian Fatalities,Number of Civilian Injured,Number of Enforcement Fatalities,Number of Enforcement Injured,Total Number of Victim Fatalities,Total Number of Victims,Date,Type of Gun - General,Total Number of Guns,Relationship to Incident Location
0,University of Texas at Austin,Austin,Texas,"30.198887300000003,-97.84415949",15,32,1,0,16,48,8/1/1966,Multiple guns,7,Place of schooling
1,Rose-Mar College of Beauty,Mesa,Arizona,"33.42268696,-111.8163202",5,1,0,0,5,6,11/12/1966,Handgun,1,None
2,New Orleans Police Shootings,New Orleans,Louisiana,"30.0687242,-89.93147412",4,8,5,5,9,22,12/31/1972,Multiple guns,2,Local government
3,Clara Barton Elementary School,Chicago,Illinois,"41.83928045,-87.68818145",1,3,0,0,1,4,1/17/1974,Handgun,2,Place of schooling
4,Olean High School,Olean,New York,"42.08185369,-78.43213922",3,7,0,0,3,10,12/30/1974,Multiple guns,2,Place of schooling
5,Los Angeles Computer Learning Center,Los Angeles,California,"34.17622092,-118.53995420000001",1,6,1,0,2,8,2/19/1976,Shotgun,1,Place of schooling
6,Cal State Fullerton,Fullerton,California,"33.88404244,-117.92785",7,2,0,0,7,9,7/12/1976,Rifle,1,Place of business/employment
7,Grover Cleveland Elementary School,San Diego,California,"32.86357277,-117.1281628",2,8,0,1,2,11,1/29/1979,Rifle,1,Place of residency
8,University of South Carolina,Columbia,South Carolina,"34.05098834,-80.82077527",2,5,0,0,2,7,10/6/1979,Handgun,1,Place of schooling
9,Valley High School,Las Vegas,Nevada,"36.18931923,-115.3264875",1,2,0,0,1,3,3/19/1982,Handgun,1,Place of schooling


In [ ]:
#ANGIE
#Create 2 new columns
shootings_df["Nearest Police Department"] = ""
shootings_df["Nearest Police Department Proximity"] = ""

#Parameters for google api nearby search 
params = {"location": "",
          "rankby": "distance", 
          "type": "police",
          "keyword": "department",
          "name": "police department",
          "key": gkey,}

#Loop through rows in the dataframe
for index, row in shootings_df.iterrows():
    
    location = row["Location"]
    
    params["location"] = location
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=params).json()
    
    #Add name of the nearest police department to column
    shootings_df.loc[index, "Nearest Police Department"] = response["results"][0]["name"]
    
    #Obtain latitude of nearest police department
    pd_lat = response["results"][0]["geometry"]["location"]["lat"]
    
    #Obtain longitude of nearest police department
    pd_lon = response["results"][0]["geometry"]["location"]["lng"]
    
    #Combine the latitude and longitude into a string
    pd_location = f"{pd_lat},{pd_lon}"
    
    #Find distance from shooting site to the nearest police department
    dist_url = ('https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial&origins={0}&destinations={1}&key={2}').format(location, pd_location, gkey)
    
    data = requests.get(dist_url).json()
    
    proximity = str(data["rows"][0]["elements"][0]["distance"]["text"])
    
    #If the unit is in ft...
    #Drop the last 3 characters
    #Convert units to miles
    #Add proximity to column 
    if " ft" in proximity:        
        proximity = proximity[:-3]
        proximity = float(proximity) / 5280
        shootings_df.loc[index, "Nearest Police Department Proximity"] = str(proximity)
      
    #Else add the proximity to column
    else:
        shootings_df.loc[index, "Nearest Police Department Proximity"] = proximity

shootings_df.head()


In [ ]:
#ANGIE
#Drop the last 3 characters in the column
shootings_df['Nearest Police Department Proximity'] = shootings_df['Nearest Police Department Proximity'].apply(lambda x: x.replace(" mi", ""))

#Convert values in column to floats
shootings_df["Nearest Police Department Proximity"] = pd.to_numeric(shootings_df["Nearest Police Department Proximity"])

#Round two decimal places
decimals = 2
shootings_df["Nearest Police Department Proximity"] = shootings_df["Nearest Police Department Proximity"].apply(lambda x: round(x, decimals))


In [ ]:
#ANGIE
#Create scatter plot
#Correlation between nearest police department proximity and total number of victims
plt.scatter(shootings_df["Nearest Police Department Proximity"], shootings_df["Total Number of Victims"], marker="o", facecolors="gold", edgecolors="black", alpha=0.65)
plt.ylabel("Total Number of Victims")
plt.xlabel("Nearest Police Department Proximity (mi)")
plt.show()

In [ ]:
#ANGIE
new_df = shootings_df

#Rename the column to remove spaces
new_df = new_df.rename(columns={"Total Number of Guns": "tot_num_guns"})

#Drop all rows that contain the unknown value
new_df = new_df[~new_df.tot_num_guns.str.contains("Unknown")]

#Convert all values in column to floats
new_df["tot_num_guns"] = pd.to_numeric(new_df["tot_num_guns"])

#Plot correlation between total number of guns and total number of victims
plt.scatter(new_df["tot_num_guns"], new_df["Total Number of Victims"], edgecolors="black", alpha=0.65)
plt.ylabel("Total Number of Victims")
plt.xlabel("Total Number of Guns Used")
plt.show()